In [49]:
%%time
import re
import numpy as np
import os
import collections
from math import log
import sys
import heapq

word2tag = dict()
tag2word = dict()
def stringNorm(sent, num=False):
    regex = re.compile(f'[,\.!?:;\'{"0-9" if not num else ""}\*\-“…\(\)„”—»«–––=\[\]’]')
    return regex.sub('',sent.lower())

def bigrams2unigrams(bigrams):
    return {w1: sum([float(bigrams[w1][w2]) for w2 in bigrams[w1]])/2 for w1 in bigrams}

with open("../data/supertags.txt") as tags:
    for line in tags:
        word, tag = stringNorm(line, num=True).split()
        word2tag[word] = tag
        if tag in tag2word:
            tag2word[tag].append(word)
        else:
            tag2word[tag] = [word]
            

bigrams =  collections.defaultdict(lambda: collections.defaultdict(int))
bigrams_rev = collections.defaultdict(lambda: collections.defaultdict(int))
unigrams = collections.defaultdict(int)
with open('../data/poleval_2grams.txt') as f:
    for x in f:
        num, w1, w2 = x.split()
        if float(num) >= 5:
            bigrams[w1][w2] = float(num)
            bigrams_rev[w2][w1] = float(num)
            unigrams[w1] += float(num) / 2
            unigrams[w2] += float(num) / 2

CPU times: user 1min 19s, sys: 1.28 s, total: 1min 20s
Wall time: 1min 19s


In [122]:
uniSum = sum((unigrams.values()))
# print(uniSum)

def PMI(w1, w2):
    return np.log(
        bigrams[w1][w2] * uniSum / 
        (unigrams[w1] * unigrams[w2]))
     
#     + sys.float_info.min)

def sameTag(w1, w2):
    if not (w1 in word2tag and w2 in word2tag):
        return 0
    if word2tag[w1] == word2tag[w2]:
        return 50
    return 10
    
        
def wordValue(word, x): 
    return np.sum([
        PMI(word, x), 
        sameTag(word,x),
#         len(x),
    ])


In [123]:
def findSimilarWords(word, k):
    return {x: wordValue(word,x)
        for x in heapq.nlargest(
            k, 
            bigrams[word], 
            key=lambda x: wordValue(word, x)
        )
    }

In [124]:
import pprint
slowa = ['kolej', 'uczelnia', 'grzyb', 'ramię', 'gitara']
nln = '\n'
for i in slowa:
    print(i)
    pprint.pprint(findSimilarWords(i, 8))
    print()

kolej
{'aglomeracyjna': 21.512572762615552,
 'aglomeracyjną': 21.094607491743155,
 'gondolowa': 21.377470277758988,
 'jednoszynowa': 22.15066016599247,
 'jednoszynową': 22.15066016599247,
 'krzesełkowa': 21.958288273345012,
 'transkontynentalna': 22.15066016599247,
 'transsyberyjska': 21.745195057884303}

uczelnia
{'kształci': 18.842583196369056,
 'kształcąca': 19.385854727042091,
 'macierzysta': 18.947316870659904,
 'niepaństwowa': 20.741058501091949,
 'niepubliczna': 19.867327383636592,
 'wypromowała': 19.626697855455703,
 'zaplanowała': 20.20060107971959,
 'łazarskiego': 19.311746754888368}

grzyb
{'atomowy': 19.87936709084363,
 'candida': 19.020625147295043,
 'jadalny': 21.939451752772626,
 'kapeluszowy': 23.439465755091643,
 'mikroskopijny': 22.714425210755742,
 'nadrzewny': 19.80942680100064,
 'niejadalny': 22.419105663323279,
 'trujący': 20.702504210495015}

ramię
{'korby': 19.362247433195215,
 'moskwy': 16.204794420953419,
 'robota': 17.511799451177588,
 'spiralne': 18.51100599

In [44]:
5 * ('ased' == 'asd')

0